In [2]:
import os
import uuid
import shutil
#import pylatex
import pandas as pd
import lightkurve as lk 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from datetime import datetime
from tqdm.notebook import tqdm
from multiprocessing import cpu_count, Pool
#from pylatex.section import Chapter, Subsection
#from pylatex import Document, Section, Figure, NoEscape, Command

%run pipeline.ipynb

import warnings
warnings.filterwarnings("ignore")

In [3]:
# bestlcs=pd.read_csv('makingtable/bestlcs.csv')
# bestlclist=np.array([], dtype=object)
# start_col=bestlcs.columns.get_loc('Sector 1')
# for index, row in bestlcs.iterrows(): #iterate through each object
#     for col in bestlcs.columns[start_col:]: #iterate through all the columns starting with the sectors
#         value=row[col]
#         if not pd.isna(value):
#             bestlclist=np.append(bestlclist, [f'{row['pop_id']}_{value}'])

In [4]:
#K2SFFlist=np.loadtxt('makingtable/K2SFFlist.txt', dtype=str)

In [5]:
#lclist=pd.concat([pd.DataFrame(bestlclist), pd.DataFrame(K2SFFlist)], ignore_index=True)

In [6]:
# lclist[['pop_id', 'lcname']]=lclist[0].str.split('_', expand=True)
# lclist.drop(columns=0, inplace=True)

In [7]:
#lclist['period']=np.NaN

In [8]:
#lclist

In [9]:
# for index, currow in lclist.iterrows(): 
#     curlc=get_lc_string(currow['lcname'])
#     try:
#         period = curlc.to_periodogram(maximum_period=5).period_at_max_power.value
#         lclist.at[index,'period']=period
#     except Exception as e:
#         print(f'Error getting period for {currow['lcname']}: {e}')

In [10]:
#lclist.to_csv('lcnameswithperiods.csv',index=False)

In [11]:
lclist=pd.read_csv('lcnameswithperiods.csv')

In [12]:
table=lclist.groupby('pop_id')['period'].agg(
    mean_per='mean',
    median_per='median',
    min_per='min',
    max_per='max',
    std_per='std'
).reset_index()

In [13]:
table['ratio']=table.std_per/table.mean_per

In [14]:
table[table.ratio>.05] #these ones all have bad periods worth investigating

,pop_id,mean_per,median_per,min_per,max_per,std_per,ratio
1,1,2.244234,3.578538,0.420936,3.631415,1.700404,0.757677
4,4,1.714242,0.781390,0.777139,3.584196,1.619429,0.944691
5,5,0.539844,0.668180,0.040876,0.910475,0.448779,0.831314
6,6,2.427288,2.968995,0.040876,4.271993,2.166950,0.892745
7,7,0.210193,0.190117,0.040881,0.399582,0.180192,0.857266
...,...,...,...,...,...,...,...
203,203,2.324719,2.324719,0.382703,4.266736,2.746426,1.181401
204,204,0.404987,0.465198,0.042916,0.466062,0.159659,0.394233
205,205,0.159881,0.159881,0.106554,0.213208,0.075416,0.471701
206,206,0.962528,0.432123,0.431019,2.554848,1.061546,1.102873


In [15]:
lclist[lclist.pop_id==3] #great period

,pop_id,lcname,period
17,3,TIC-175594111-91-SPOC-120,0.599792
18,3,EPIC-203354381-2-EVEREST-1800,0.599550
930,3,EPIC-203354381-2-K2SFF-1800,0.599227


In [16]:
lclist[lclist.pop_id==1] #weird because it was two different estimates but other than that it's well grouped

,pop_id,lcname,period
6,1,TIC-59091144-5-FFI-30min,3.609511
7,1,TIC-59091144-32-FFI-10min,0.437500
8,1,TIC-59091144-43-SPOC-120,0.421091
9,1,TIC-59091144-44-SPOC-120,0.420936
10,1,TIC-59091144-71-FFI-200s,3.578538
11,1,EPIC-246682490-13-EVEREST-1800,3.610645
928,1,EPIC-246682490-13-K2SFF-1800,3.631415


In [17]:
lclist[lclist.pop_id==105] #pretty good but still unclear what would actually be best to put into a table for third decimal

,pop_id,lcname,period
405,105,TIC-177309964-1-SPOC-120,0.452776
406,105,TIC-177309964-2-SPOC-120,0.453973
407,105,TIC-177309964-3-SPOC-120,0.454087
408,105,TIC-177309964-4-SPOC-120,0.453956
409,105,TIC-177309964-5-SPOC-120,0.453194
410,105,TIC-177309964-6-SPOC-120,0.454022
411,105,TIC-177309964-7-SPOC-120,0.453916
412,105,TIC-177309964-8-SPOC-120,0.453231
413,105,TIC-177309964-9-SPOC-120,0.452718
414,105,TIC-177309964-10-SPOC-120,0.453417


In [18]:
lclist[lclist.pop_id==106] #mostly good but some bad

,pop_id,lcname,period
444,106,TIC-300651846-1-FFI-30min,0.343759
445,106,TIC-300651846-2-FFI-30min,0.344200
446,106,TIC-300651846-3-FFI-30min,0.123886
447,106,TIC-300651846-4-FFI-30min,0.343544
448,106,TIC-300651846-5-FFI-30min,0.343896
449,106,TIC-300651846-6-FFI-30min,0.344422
450,106,TIC-300651846-7-FFI-30min,0.343497
451,106,TIC-300651846-8-FFI-30min,0.057316
452,106,TIC-300651846-9-FFI-30min,0.343870
453,106,TIC-300651846-10-FFI-30min,0.343750


In [48]:
lclist[lclist.pop_id==7] #total garbage

,pop_id,lcname,period
25,7,TIC-49072162-91-SPOC-120,0.190117
26,7,EPIC-204060981-2-EVEREST-1800,0.040881
934,7,EPIC-204060981-2-K2SFF-1800,0.399582


In [84]:
dtypes={'pop_id':'string','TIC':'string','gaiadr3_source_id':'string','epic_id':'string'}
id_table=pd.read_csv('makingtable/idswithdiscopaper.csv',dtype=dtypes)

In [94]:
table['pop_id']=table['pop_id'].astype(str)

In [96]:
table=pd.merge(table,id_table,on='pop_id')

In [108]:
table['percent_diff']=np.abs(table.mean_per-table.per)/((table.mean_per+table.per)/2)

In [114]:
table[table.percent_diff>.05] #pretty similar list to above

,pop_id,mean_per,median_per,min_per,max_per,std_per,ratio,TIC,gaiadr3_source_id,epic_id,per,group,age_Myr,disco_paper,author,year,percent_diff
1,1,2.244234,3.578538,0.420936,3.631415,1.700404,0.757677,59091144,3392553852037339776,246682490,0.437700,Taurus,2.0,Stauffer_2018,Stauffer,2018,1.347187
4,4,1.714242,0.781390,0.777139,3.584196,1.619429,0.944691,322928171,6048612514663537280,203636498,0.779400,Upper Scorpius,10.0,Stauffer_2018,Stauffer,2018,0.749780
5,5,0.539844,0.668180,0.040876,0.910475,0.448779,0.831314,175743566,6049078741946841856,203821589,0.667700,Rho Ophiuchi,1.0,Stauffer_2018,Stauffer,2018,0.211763
6,6,2.427288,2.968995,0.040876,4.271993,2.166950,0.892745,98689067,6049136569386567296,203897692,0.604300,Rho Ophiuchi,1.0,Stauffer_2018,Stauffer,2018,1.202662
7,7,0.210193,0.190117,0.040881,0.399582,0.180192,0.857266,49072162,6049979104536677632,204060981,0.399600,Upper Scorpius,10.0,Stauffer_2018,Stauffer,2018,0.621216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,203,2.324719,2.324719,0.382703,4.266736,2.746426,1.181401,301676454,3795298673460008064,<NA>,0.382500,Argus,45.0,Bouma_2024,Bouma,2024,1.434844
204,204,0.404987,0.465198,0.042916,0.466062,0.159659,0.394233,58084670,2034818379464853376,<NA>,0.465000,Field,NaN,Bouma_2024,Bouma,2024,0.137962
205,205,0.159881,0.159881,0.106554,0.213208,0.075416,0.471701,67745212,3191994547355916288,<NA>,0.213333,Columba,42.0,Bouma_2024,Bouma,2024,0.286444
206,206,0.962528,0.432123,0.431019,2.554848,1.061546,1.102873,5714469,3493814268751183744,<NA>,0.431250,Upper Centaurus-Lupus,16.0,Bouma_2024,Bouma,2024,0.762357


In [126]:
table['percent_diff2']=np.abs(table.median_per-table.per)/((table.mean_per+table.per)/2)

In [130]:
table[table.percent_diff2>.05].shape #pretty similar list to above

(59, 18)